## Connection to the Azure SQL Database

Defined some variables to programmatically create the connection to the SQL Database.

In [ ]:
jdbcUsername = "feuplogin"
jdbcPassword = "Logproject33"
jdbcHostname = "intranetfeupserver.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "intranet14"

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;".format(jdbcHostname, jdbcPort, jdbcDatabase)

connectionProperties = {
  "user": jdbcUsername,
  "password": jdbcPassword,
  "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

## Data Transformation Test

To test the data transformation executed, I reduced the dataset to the project AGEAS.2017.190 (ID: 155)

In [ ]:
# PROJECT FACT

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Produtividade").getOrCreate()

# compare hours planned
hours_planned_stg = spark.read.jdbc(url=jdbcUrl, table="(select sum(H.[NHoras]) as [HorasPrevistas] \
                                                            from [stg].[EXT_TBL_PROJETOS] as P \
                                                            join [stg].[EXT_TBL_HORASPREVISTAS] as H on P.[CodigoProjeto] = H.[CodigoProjeto] \
                                                            where P.[CodigoProjeto] = 'AGEAS.2017.190' \
                                                            group by P.[CodigoProjeto]) AS query", properties=connectionProperties).collect()
hours_planned_dw = spark.read.jdbc(url=jdbcUrl, table="(select Horas_Previstas_Projeto from [dwProdutividade].[DIM_PROJETO] where Codigo_Projeto = 'AGEAS.2017.190') AS query", properties=connectionProperties).collect()

print("HOURS PLANNED - AGEAS.2017.190")
print("Hours Planned STG: " + str(hours_planned_stg[0][0]))
print("Hours Planned DW: " + str(hours_planned_dw[0][0]))
#print(hours_planned_stg[0][0] == hours_planned_dw[0][0])

print('----------------------------------------------\n')

# compare total hours executed
hours_performed_stg = spark.read.jdbc(url=jdbcUrl, table="(select sum([NHoras]) as Total_Horas_Realizadas_Projeto \
                                                            from [stg].[EXT_TBL_IMPUTACAO_DETALHE] \
                                                            where [CodigoProjeto] = 'AGEAS.2017.190' \
                                                            group by [CodigoProjeto]) AS query", properties=connectionProperties).collect()
hours_performed_dw = spark.read.jdbc(url=jdbcUrl, table="(select sum([Horas_Realizadas_Projeto]) as Total_Horas_Realizadas_Projeto \
                                                            from [dwProdutividade].[FACTO_PROJETO] \
                                                            where [ID_Projeto] = 155) AS query", properties=connectionProperties).collect()

print("TOTAL HOURS PERFORMED - AGEAS.2017.190")
print("Hours PERFORMED STG: " + str(hours_performed_stg[0][0]))
print("Hours PERFORMED DW: " + str(hours_performed_dw[0][0]))
#print(hours_planned_stg[0][0] == hours_planned_dw[0][0])

print('----------------------------------------------\n')

# Imputations - By Project

print("IMPUTACAO_DETALHE")
imputations_dw = spark.read.jdbc(url=jdbcUrl, table="(select IMP.CodigoProjeto, IMP.Ano, IMP.Mes, IMP.SumNHoras as Horas_Realizadas, HA.Avanco, IMP.Estado \
                                                        from \
                                                            (select IM.CodigoProjeto as CodigoProjeto, IM.Ano as Ano, IM.Mes as Mes, sum(IM.NHoras) as SumNHoras, P.Estado as Estado \
                                                            from stg.EXT_TBL_IMPUTACAO_DETALHE as IM, stg.EXT_TBL_PROJETOS as P \
                                                            where IM.CodigoProjeto = 'AGEAS.2017.190' and IM.CodigoProjeto = P.CodigoProjeto \
                                                            group by IM.CodigoProjeto, IM.Ano, IM.Mes, P.Estado) as IMP \
                                                        left join stg.EXT_TBL_HISTORICO_AVANCOS as HA \
                                                        on IMP.CodigoProjeto = HA.CodigoProjecto and Ano = YEAR(DataAvanco) and Mes = MONTH(DataAvanco) \
                                                        group by IMP.CodigoProjeto, IMP.Ano, IMP.Mes, IMP.SumNHoras, HA.Avanco, IMP.Estado \
                                                        order by IMP.Ano ASC, IMP.Mes ASC) AS query", properties=connectionProperties)

print("FACTO PROJETO")
imputations_dw = spark.read.jdbc(url=jdbcUrl, table="(select Codigo_Projeto, ID_Calendario, Horas_Realizadas_Projeto as Horas_Realizadas, Avanco_Projeto as Avanco, \
                                                        Horas_Previstas_Mes_Por_Avanco, Horas_Previstas_Projeto\
                                                        from dwProdutividade.FACTO_PROJETO as FP, dwProdutividade.DIM_PROJETO as P, dwProdutividade.DIM_ESTADO as E \
                                                        where FP.ID_Projeto = 155 \
                                                            and FP.ID_Projeto = P.ID_Projeto \
                                                            and FP.ID_Estado = E.ID_Estado) AS query", properties=connectionProperties)

In [ ]:
# TASK FACT

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Produtividade").getOrCreate()


# compare hours planned
hours_planned_stg = spark.read.jdbc(url=jdbcUrl, table="(select [ID], [CodigoProjeto], [Username], [Tarefa], [NHoras] \
                                                            from [stg].[EXT_TBL_HORASPREVISTAS] \
                                                            where CodigoProjeto = 'AGEAS.2017.190') AS query", properties=connectionProperties)
hours_planned_dw = spark.read.jdbc(url=jdbcUrl, table="(select ID_Projeto, FT.ID_Tarefa, Horas_Previstas_Tarefa \
                                                            from dwProdutividade.Facto_Tarefa as FT, dwProdutividade.DIM_TAREFA as DT \
                                                            where ID_Projeto = 155 and FT.ID_Tarefa = DT.ID_Tarefa \
                                                            group by ID_Projeto, FT.ID_Tarefa, Horas_Previstas_Tarefa) AS query", properties=connectionProperties)

imputations_proof = spark.read.jdbc(url=jdbcUrl, table="(select [CodigoProjeto], [Username], [FK_TarefaID], sum([NHoras]) as Horas_Realizadas_Tarefa \
                                                            from [stg].[EXT_TBL_IMPUTACAO_DETALHE] \
                                                            where CodigoProjeto = 'AGEAS.2017.190' \
                                                            group by [CodigoProjeto], [Username], [FK_TarefaID]) AS query", properties=connectionProperties)

print("HOURS PLANNED - AGEAS.2017.190 TASKS")
print("Hours Planned STG:\n")
hours_planned_stg.show()
print("Hours Planned DW: \n")
hours_planned_dw.show()
print("Proof that the FKTarefaID is null: \n")
imputations_proof.show()

print('----------------------------------------------\n')

# Imputations - By Task
hours_performed_stg = spark.read.jdbc(url=jdbcUrl, table="(select [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID], sum([NHoras]) as Horas_Realizadas_Tarefa \
                                                            from [stg].[EXT_TBL_IMPUTACAO_DETALHE] \
                                                            where CodigoProjeto = 'AGEAS.2017.190' \
                                                            group by [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID] \
                                                            order by [Ano] ASC, [Mes] ASC) AS query", properties=connectionProperties)

query = "(select Codigo_Projeto, Nome_Funcionario, ID_Calendario, ID_Tarefa, Horas_Realizadas_Tarefa, ID_Perfil, ID_Classificacao_Produtividade_Tarefa \
from dwProdutividade.FACTO_TAREFA AS FT, dwProdutividade.DIM_PROJETO AS DP, dwProdutividade.DIM_FUNCIONARIO as DF \
where FT.ID_Projeto = 155 and FT.ID_Projeto = DP.ID_Projeto and FT.ID_Funcionario = DF.ID_Funcionario \
order by ID_Calendario ASC) AS query"
hours_performed_dw = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)

hours_performed_stg.show()
hours_performed_dw.show()